In [1]:
from collections import OrderedDict

In [2]:
import requests
import json,sys
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import binascii
import pandas as pd

def flushprint(d):
    sys.stdout.write('\r')
    sys.stdout.write(str(d))
    sys.stdout.flush()    
    
headers={
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'close',
    #'Connection': 'keep-alive',
    'Host':'music.163.com',
    'Referer': 'http://music.163.com/',
    'Upgrade-Insecure-Requests': '1',
    'Cookie': 'appver=1.5.0.75771',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36',
}

In [3]:
def aesEncrypt(text, secKey):
    """
    aes加密
    :param text: 要加密的文本
    :param pubKey:密钥
    :return:返回加密过后的文本（aes加密完后在进行一遍base64）
    """
    blockSize = 16     
    encryptor = AES.new(secKey.encode(), AES.MODE_CBC, b'0102030405060708')
    text = pad(text.encode(),blockSize) #将texe的长度填充到16的倍数，不然后面的加密回报错
    ciphertext = encryptor.encrypt(text) # 进行AES加密
    ciphertext = base64.b64encode(ciphertext) #把加密后的结果再进行一遍base64
    return str(ciphertext,encoding='utf-8') # 由于上面得到的是一个bytes 所以将它转换为字符串
def rsaEncrypt(text, pubKey, modulus):
    """
    将密钥进行rsa加密
    :param text: 16位随机字符
    :param pubKey:公钥
    :param modulus:modulus
    :return:
    """
    text = text[::-1]
    rs = int(binascii.b2a_hex(text.encode()), base=16) ** int(pubKey, base=16) % int(modulus, base=16)
    return format(rs, 'x').zfill(256)
def createSecretKey(size):
    """
    返回一个长度为size的随机字符串 这里直接返回n个F了
    :param size: 生成字符串的长度
    :return:返回一个长度为size的随机字符串
    """
    return 'F' * size

In [4]:
def userlisten(uid,listenlist):
#post请求地址 这里为得到一首歌曲的url地址的api 其他api看后面
    url = 'http://music.163.com/weapi/v1/play/record?csrf_token='
#   headers = {'Referer': 'http://music.163.com/'}
   #这个不知道是什么东西 跟着抄过来
    modulus = '00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7'
    #将json进行第一次aes加密的时候用这个作为密钥
    nonce = '0CoJUm6Qyw8W8jud'# rsa加密的公钥
    pubKey = '010001'#构建json 这是获得一首歌曲的url的api 其他的api看后面
    
    
    text1 = {
            "uid":str(int(uid)),#用户id
            "type":"1", 
            #为1时获得该用户一个星期听歌排行，为0时获得所有时间听歌排行
            "limit":"1000",
            "offset":"0",
            "total":"true",
            "csrf_token":""
           }
         # 将字典dump成一个json字符串
    text1 = json.dumps(text1)
    #获得一个16位的随机字符串，我们这里图省事给的是16个F
    secKey1 = createSecretKey(16)
    encText1 = aesEncrypt(text1,nonce) #把上面构建的json数据使用nonce进行第一次aes加密
    encText1 = aesEncrypt(encText1,secKey1)#使用随机的十六位字符串作为密钥把上面的到的结果进行第二次加密
    encSecKey1 = rsaEncrypt(secKey1, pubKey, modulus) #把我们用作ARS加密的密钥进行ras加密
    #由于我们这里使用的一直是使用十六个F作为密钥所以这个对于我们来说是不变的，所以只要加密一遍以后直接复制出来搞成一个常量就行，每次提交给服务器都一样
    data1 = {
            'params': str(encText1),
            'encSecKey': encSecKey1
           }
    try:
        req1 = requests.post(url, headers=headers, data=data1) #发送post请求
        info1 = json.loads(req1.text)
        #if info1['code']==200:
        #    weekinfo = info1
        #else:
        #    print('  ',uid,'  ',info1)
        weekinfo = info1
    except Exception as e:
        print('  ',uid,'  ',e)
        pass
    
    
    
    listen=[uid,weekinfo]
    #print(listen)
    listenlist.append(listen)
    dict(listen)

In [ ]:
#path = 'comment_and_user_informations_with_albums.csv'
#data = pd.read_csv(path,sep='\t',encoding='utf-8',error_bad_lines=False,lineterminator='\n',
#                   names=['song_id', 'total', 'userid', 'username', 'commentId', 'content',
#           'likedCount', 'ishotcom', 'time', 'berepid', 'berepcontent',
#           'berepuser', 'berepusername', 'gender', 'birthday', 'city', 'province',
#           'nickname', 'signature', 'userType', 'vipType', 'level', 'followeds',
#           'follows', 'createDays', 'createTime', 'listenSongs', 'eventCount',
#           'playlistBeSubscribedCount', 'playlistCount', 'allSubscribedCount',
#           'mobileSign', 'pcSign', 'peopleCanSeeMyPlayRecord', 'binurl',
#           'music_name', 'album_id', 'album_publishTime'])

In [ ]:
#data[data.ishotcom == 1.0]

In [ ]:
#cf = data[data.ishotcom == 1.0].drop_duplicates('userid', keep='first', inplace=False).reset_index(drop=True)

In [5]:
#ipath='userid_for_crawl_listenlist.csv'
#从保存的文件中读取内容
data = pd.read_csv('test.csv')

In [6]:
ids=data['userid']
ids

0    304750847
1    304774790
2    304820633
3    304835457
4    304863751
Name: userid, dtype: int64

In [7]:
mid=0
rids=ids[mid:5]

In [8]:
listenlist=[]

In [9]:
opath = 'listenlist_test.json'
opath

'listenlist_test.json'

In [10]:
for k,i in enumerate(rids):
    userlisten(i,listenlist)
    flushprint(k)
    if k%20 ==0 and k!=0 :
        data =json.dumps(listen) 
        with open(opath,'a+',encoding='utf-8') as f:
            listen.to_json(orient='records')
        listenlist.clear()
    if k%100 ==0 and k!=0:
        num= k+mid+1
        print('save the user of ',str(num), ' / 50000 ; ','%.2f%%' % (num/50000*100))

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [ ]:
import pandas as pd

dat=pd.DataFrame(listenlist)

In [ ]:
dat.shape
dat
dat.to_csv('df.csv')

In [ ]:
result = dat.to_json(orient="records")
parsed = json.loads(result)
d=json.dumps(parsed, indent=4)  

In [ ]:
d

In [ ]:
with open(opath,'a+',encoding='utf-8') as f:
    data.to_csv(f,sep='\t',encoding='utf-8',header=0,index=False)

## 案例草稿

In [ ]:
#case 1 一切正常的
uid=1671165524
userlisten(uid,listenlist)

In [ ]:
#case2 有隐私权限的
uid=1430350262
userlisten(uid,listenlist)

In [ ]:
uid=100167517
userlisten(uid,listenlist)

In [ ]:
uid=74699896
userlisten(uid,listenlist)

In [ ]:
pd.DataFrame(listenlist,columns=['user_id','week','alltime'])

In [ ]:
uid=1430350262
weeklisten(uid)

## 调试

In [ ]:
def weeklisten(uid):
#post请求地址 这里为得到一首歌曲的url地址的api 其他api看后面
    url = 'http://music.163.com/weapi/v1/play/record?csrf_token='
    headers = {
    'Referer': 'http://music.163.com/'
    }
    #这个不知道是什么东西 跟着抄过来
    modulus = '00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7'
    #将json进行第一次aes加密的时候用这个作为密钥
    nonce = '0CoJUm6Qyw8W8jud'# rsa加密的公钥
    pubKey = '010001'#构建json 这是获得一首歌曲的url的api 其他的api看后面
    
    text1 = {
            "uid":str(int(uid)),#用户id
            "type":"1", 
            #为1时获得该用户一个星期听歌排行，为0时获得所有时间听歌排行
            "limit":"1000",
            "offset":"0",
            "total":"true",
            "csrf_token":""
           }
         # 将字典dump成一个json字符串
    text1 = json.dumps(text1)
    #获得一个16位的随机字符串，我们这里图省事给的是16个F
    secKey1 = createSecretKey(16)
    encText1 = aesEncrypt(text1,nonce) #把上面构建的json数据使用nonce进行第一次aes加密
    encText1 = aesEncrypt(encText1,secKey1)#使用随机的十六位字符串作为密钥把上面的到的结果进行第二次加密
    encSecKey1 = rsaEncrypt(secKey1, pubKey, modulus) #把我们用作ARS加密的密钥进行ras加密
    #由于我们这里使用的一直是使用十六个F作为密钥所以这个对于我们来说是不变的，所以只要加密一遍以后直接复制出来搞成一个常量就行，每次提交给服务器都一样
    data1 = {
            'params': str(encText1),
            'encSecKey': encSecKey1
           }
    req1 = requests.post(url, headers=headers, data=data1) #发送post请求
    info1 = json.loads(req1.text)

    if info1['code']==200:
        weekinfo = info1
        return weekinfo
    else:
        print('  ',uid,'  ',info1)
        weekinfo = info1

In [ ]:
uid =74699896
weeklisten(uid)

In [ ]:
uid =1430350262
weeklisten(uid)

In [ ]:
len(weekinfo)